In [1]:
from fredapi import Fred
import warnings
warnings.filterwarnings('ignore')
from fred import Fred
import pandas as pd
import re
import time
from functools import reduce

In [2]:
fr = Fred(api_key='f13196245a409ee3db300071145818eb',response_type='dict')
all_tags=pd.DataFrame(fr.tag.tags())
tag_name=all_tags['name']
related_tag=pd.DataFrame(fr.tag.related_tags('al'))
tag=related_tag['name'].sort_values()
#tag.to_csv('tag.csv')
#tag_name.sort_values()[150:200]

In [62]:
state_list=["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware",'District of Columbia',"Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
states=[x.lower() for x in states]

In [115]:
def get_series_df(series_id,title,frequency,state):
    df=pd.DataFrame(fr.series.observations(series_id))
    df=df.loc[:,['date','value']]
    df.columns=['date',title]
    df['year']=[x.year for x in df['date']]
    if frequency=='A':
        df=df.loc[:,['year',title]]
    if frequency=='Q':
        df['month']=[x.month for x in df['date']]
        df=df.loc[df['month']==1,:]
        df=df.loc[:,['year',title]]
    if frequency=='M':
        df['month']=[x.month for x in df['date']]
        df=df.loc[df['month']==1,:]
        df=df.loc[:,['year',title]] 
    if frequency=='W':
        df=df.groupby(['year'])[title].mean().reset_index()
        df.columns=['year',title]
    df['state']=state
    df=df.loc[df['year']>=1992,:]
    return df

def merge_id_df(df):
    df=df.reset_index(drop=True)
    state_count=0
    for i in range(df.shape[0]):
        temp_id=df.loc[i,'id']
        temp_df=get_series_df(temp_id,df.loc[i,'m_id'],df.loc[i,'frequency_short'],df.loc[i,'state'])
        if state_count==0:
            state_df=temp_df
        else:
            state_df=state_df.append(temp_df)
        state_count=state_count+1   
    #print(state_df.head(2))
    state_df.columns=['year',df.loc[i,'title'],'state']
    return state_df

In [20]:
tag_list=['exports']

In [36]:
tag_list1=['households','housing','infrastructure',
          'internet','investment','jobs',
          'legal','manufacturing','military',
          'nondurable goods','private','railroad']

In [54]:
tag_list2=['cost','crime','deaths',
          'family','food','fuels',
          'machines','recreation','retail',
          'retail trade','sale','sales taxes',
          'single-parent','subprime','violent crime']

In [21]:
import re
temp_tag_df_list=[]
tag_df_list=[]
divide_count=1
for tag in tag_list:
    state_count=0
    for state in states:
        #print(temp_search)
        temp_search=state+';'+tag
        try:
            temp_df=pd.DataFrame(fr.tag.series(temp_search,order_by='popularity'))
        except:
            continue
        if temp_df.shape[0]==0:
            continue
        temp_df['state']=state
        temp_df['m_id']=[re.sub(state.upper(),'',x) for x in temp_df['id'] ]
        temp_df=temp_df.loc[~temp_df['title'].str.contains(
                'Gross Domestic Product|All Employees|Consumer Price Index|Personal Consumption|Number of Identified Exporters|Wages and Salaries'),:]
        if state_count==0:
            state_id_df=temp_df
        else:
            state_id_df=state_id_df.append(temp_df)
        state_count+=1
    if divide_count%3!=0:    
        temp_tag_df_list.append(state_id_df)
    else:
        temp_tag_df_list.append(state_id_df)
        tag_df_list.append(temp_tag_df_list)
        temp_tag_df_list=[]
    divide_count+=1

In [ ]:
scrapy_data_list=[]
tag_count=0
for temp_df_list in tag_df_list:
    for df in temp_df_list:
        print(tag_list[tag_count])
        tag_count+=1
        get_id=df['m_id'].value_counts()
        get_id=get_id[get_id>=30]
        id_list=list(get_id.index)
        temp_df=df.loc[df['m_id'].isin(id_list)]
        print(temp_df.shape)
        scrapy_data_list.append(temp_df)
scrapy_final = reduce(lambda left,right: left.append(right),scrapy_data_list)
scrapy_final=scrapy_final.reset_index(drop=True)

In [39]:
grouped=scrapy_final.groupby('m_id')
group_count=0
for name, group in grouped:
    if group_count==0:
        final_df=merge_id_df(group)
    else:
        temp_df=merge_id_df(group)
        temp_df=temp_df.drop_duplicates()
        final_df=pd.merge(final_df,temp_df,on=['state','year'],how='outer')
        final_df=final_df.drop_duplicates()
    group_count+=1
final_df.columns=[re.sub('for Wyoming|in Wyoming','',x) for x in final_df.columns]
#final_df.dropna(axis=1,thresh=1200)

        
    

In [48]:
column_to_use=['year',
       'state',
       'New Private Housing Units Authorized by Building Permits: 1-Unit Structures _y',
       'New Private Housing Units Authorized by Building Permits _x',
       'Durable Manufacturing Earnings ', 'Nondurable Manufacturing Earnings ',
       'Homeownership Rate ', 'Home Vacancy Rate ', 'Median Household Income ',
       'All-Transactions House Price Index ',
       'Total Investment Securities at Book Value for Commercial Banks  (DISCONTINUED)',
       'Job Losers and Persons Who Completed Temporary Jobs, as a Percent of the Civilian Labor Force ',
       'Durable Manufacturing Wages and Salaries ',
       'Nondurable Manufacturing Wages and Salaries ']

In [50]:
final_df=final_df.loc[:,column_to_use]
print(final_df.shape)
final_df_name=('_').join(tag_list1)
final_df.to_csv(final_df_name+'.csv',index=False)

In [86]:
test=pd.DataFrame(fr.series.observations('CAICLAIMS'))

In [96]:
test['month']=[x.month for x in test['date']]
test['year']=[x.year for x in test['date']]

In [28]:
series_list=['PHCI','ICLAIMS','SLIND','POP','NIM','SLGRTAX','NP','CCLAIMS','HICCOVPCT','WLDA052SCEN','NUM','TISBV']
series_list1=['STTMINWG','UNEMPLOY']
series_list2=['CCDIOAOHQ156N','CHEXMOH39A647NCEN','M02110US000OHM398NNBR']

In [65]:
def generate_scrapy_df_by_series(series_id,previous_id):
    scrapy_state_list=[]
    for state in states:
        if previous_id==1:
            temp_state_id=state.upper()+series_id
        else:
            temp_state_id=series_id+state.upper()
        scrapy_state_list.append(temp_state_id)
    count=0
    temp_state=state_list.copy()
    for state_id in scrapy_state_list:
        try:
            temp_df=pd.DataFrame(fr.series.details(state_id))
            temp_df=temp_df.loc[:,['frequency_short','id','title']]
            if count==0:
                final_df=temp_df
            else:
                final_df=final_df.append(temp_df) 
        except:
            print(state_id)
            temp_state.remove(state_list[count])
        count=count+1
    final_df['state']=temp_state
    return final_df    
def merge_series_df(df):
    df=df.reset_index(drop=True)
    state_count=0
    final_title=df.loc[0,'title']
    final_title=re.sub('Alabama','',final_title)
    for i in range(df.shape[0]):
        temp_id=df.loc[i,'id']
        temp_df=get_series_df(temp_id,final_title,df.loc[i,'frequency_short'],df.loc[i,'state'])
        if state_count==0:
            state_df=temp_df
        else:
            state_df=state_df.append(temp_df)
        state_count=state_count+1   
    return state_df

In [127]:
count=0
scrap_df_list=[]
for m_id in series_list:
    temp_scrap_df=generate_scrapy_df_by_series(m_id,1)
    scrap_df_list.append(temp_scrap_df)

    

(50, 3)
50
DCSLIND
51
(50, 3)
50
(51, 3)
51
(51, 3)
51
DCSLGRTAX
51
(50, 3)
50
(51, 3)
51
WICCLAIMS
51
(50, 3)
50
(51, 3)
51
(51, 3)
51
(51, 3)
51
(51, 3)
51


In [122]:
count=0
for temp_scrap_df in scrap_df_list:    
    temp_series_df=merge_series_df(temp_scrap_df)
    temp_series_df=temp_series_df.drop_duplicates()
    if count==0:
        final_df=temp_series_df
    else:
        final_df=pd.merge(final_df,temp_series_df,on=['state','year'],how='outer')
    count=count+1

In [124]:
final_df.to_csv('extra_feature1.csv',index=False)

In [126]:
final_df.head(5)

,year,Coincident Economic Activity Index for,state,Initial Claims in,Leading Index for,Resident Population in,Net Interest Margin for Banks in,"State Government Tax Collections, Total Sales and Gross Receipts Taxes in",Total Nonperforming Loans for Commercial Banks in,Continued Claims (Insured Unemployment) in,Health Insurance Coverage: Coverage Rate in (DISCONTINUED),Value of Exports to All Countries from,Commercial Banks in,Total Investment Securities at Book Value for Commercial Banks in (DISCONTINUED)
0,1992,68.67,Alabama,6830.442308,0.59,4139.269,4.71,2218884.0,293436,38284.403846,NaN,3.279424e+09,218,11553259.0
1,1993,70.09,Alabama,6887.307692,0.57,4193.114,4.83,2514799.0,237026,33402.403846,NaN,NaN,216,12051861.0
2,1994,72.70,Alabama,6004.924528,2.10,4232.965,5.24,2559992.0,216044,31312.056604,NaN,NaN,211,NaN
3,1995,75.75,Alabama,6809.673077,1.33,4262.731,4.52,2645405.0,190362,30748.634615,NaN,NaN,185,NaN
4,1996,78.29,Alabama,6647.384615,1.48,4290.403,4.25,2773974.0,239862,33069.519231,NaN,NaN,185,NaN


(1377, 14)


In [64]:
'''
total_df_list=[]
for tag in tag_list:
    state_count=0
    for state in states:
        temp_search=state+';'+tag
        #print(temp_search)
        temp_df=pd.DataFrame(fr.tag.series(temp_search,order_by='popularity'))
        temp_df=temp_df.loc[:,['id']]
        temp_df['m_id']=[re.sub(state.upper(),'',x) for x in temp_df['id'] ]
        if state_count==0:
            final_df=temp_df
        else:
            final_df=pd.merge(final_df,temp_df,on='m_id',how='inner')
        state_count+=1
    total_df_list.append(final_df)
'''


'''
albama=fr.category.series('27282',order_by='popularity',sort_order='desc')
alska=fr.category.series('27286',order_by='popularity',sort_order='desc')
#help(fr.category.series)
albama=pd.DataFrame(albama)
alska=pd.DataFrame(alska)
albama=albama.loc[:,['id','title']]
alska=alska.loc[:,['id','title']]
import re
albama['title']=[re.sub('for Alabama|in Alabama','',x) for x in albama['title'] ]
alska['title']=[re.sub('for California|in California','',x) for x in alska['title'] ]
albama['id_me']=[re.sub('AL','',x) for x in albama['id'] ]
alska['id_me']=[re.sub('CA','',x) for x in alska['id'] ]
'''

"\nalbama=fr.category.series('27282',order_by='popularity',sort_order='desc')\nalska=fr.category.series('27286',order_by='popularity',sort_order='desc')\n#help(fr.category.series)\nalbama=pd.DataFrame(albama)\nalska=pd.DataFrame(alska)\nalbama=albama.loc[:,['id','title']]\nalska=alska.loc[:,['id','title']]\nimport re\nalbama['title']=[re.sub('for Alabama|in Alabama','',x) for x in albama['title'] ]\nalska['title']=[re.sub('for California|in California','',x) for x in alska['title'] ]\nalbama['id_me']=[re.sub('AL','',x) for x in albama['id'] ]\nalska['id_me']=[re.sub('CA','',x) for x in alska['id'] ]\n"

In [ ]:
for i in datasets.iloc[:,0]:
    if i[0:3]=='BDS':
        print(i)
dataset = 'BDSFirmsTimeSeries'
con = cen.base.Connection(dataset)
con.geographies['fips']
con.query(cols=['estabs'],geo_unit='state',apikey='879116b83035eef6bc66700515158f25b5f38c57')
state_category=pd.DataFrame(fr.category.children('27281'))
state_category=state_category.loc[state_category['name'].isin(state_list),:]
state_category.reset_index(drop=True)